## Extensive data
 I created a larger monet dataset by uploading an image from wikiart. And then convert it to TFRecords. Hope it helps your work

Dataset: https://www.kaggle.com/doanquanvietnamca/monet-ext-data
![monet](https://camo.githubusercontent.com/1862adecd202ba420847653d9a119f9fed9a3abd/68747470733a2f2f6a756e79616e7a2e6769746875622e696f2f4379636c6547414e2f696d616765732f70686f746f327061696e74696e672e6a7067)

## Script to download from WikiArt
```
def provide_monet_dataset():
    # Dowload the script that will manage the data procurement
    # https://github.com/lucasdavid/wikiart
    os.system("git clone https://github.com/lucasdavid/wikiart.git")
    os.system("python3 wikiart/wikiart.py --datadir ./wikiart-saved/ fetch --only artists")

    # Metadata to indicate the artist to be downloaded
    entry = '''[{
        "artistName": "Claude Monet",
        "birthDay": "/Date(-4074969600000)/",
        "birthDayAsString": "November 14, 1840",
        "contentId": 211667,
        "deathDay": "/Date(-1359331200000)/",
        "deathDayAsString": "December 5, 1926",
        "dictonaries": [
            1221,
            316
        ],
        "image": "https://uploads0.wikiart.org/00115/images/claude-monet/440px-claude-monet-1899-nadar-crop.jpg!Portrait.jpg",
        "lastNameFirst": "Monet Claude",
        "url": "claude-monet",
        "wikipediaUrl": "http://en.wikipedia.org/wiki/Claude_Monet"
    }]'''

    with open('./wikiart-saved/meta/artists.json', 'w') as artists_file:
        artists_file.write(entry)

    # Download the art from the artist
    os.system("python3 wikiart/wikiart.py --datadir ./wikiart-saved/ fetch")

    # Filepath of all files retrieval
    files = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk('wikiart-saved/images/claude-monet'):
        for file in f:
            if '.jpg' in file:
                files.append(os.path.join(r, file))

    images = []

    for file in files:
        images.append(_preprocess(cv2.imread(file)))

    # An array with al the read fieles is returned
    return images
```

In [ ]:
# LOAD LIBRARIES
import numpy as np, pandas as pd, os
import matplotlib.pyplot as plt, cv2
import tensorflow as tf, re, math

In [ ]:
!ls ../input/oregon-wildlife/oregon_wildlife/oregon_wildlife/

In [ ]:
# PATHS TO IMAGES
PATH = '../input/oregon-wildlife/oregon_wildlife/oregon_wildlife/bald_eagle'
IMGS = os.listdir(PATH)
print('There are %i monet images '%(len(IMGS)))

In [ ]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:
def serialize_example(feature0, feature1, feature2):
  feature = {
      'image_name': _bytes_feature(feature0),
      'image': _bytes_feature(feature1),
      'target': _bytes_feature(feature2)
  }
  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()

## Write TFRecords

In [ ]:

PATH = '../input/oregon-wildlife/oregon_wildlife/oregon_wildlife/'

for dirr in os.listdir(PATH):
    log = 0
    IMGS = os.listdir(PATH+ dirr)
    with tf.io.TFRecordWriter(f'wild_{dirr}.tfrec') as writer:
        for k in range(len(IMGS)):
            try:
                img = cv2.imread(f"../input/oregon-wildlife/oregon_wildlife/oregon_wildlife/{dirr}/"+IMGS[k])

                h, w, *_ = img.shape 
                if h*w > 1024*1024:
                    img = cv2.imencode('.jpg', img, (cv2.IMWRITE_JPEG_QUALITY, 94))[1].tostring()
                    name = IMGS[k].split('.')[0]
                    example = serialize_example(
                        str.encode(name),
                        img, 
                        b'monet')
                    writer.write(example)
                    log+=1
                if k%100==0: print(log,', ', end='')
            except:
                pass
    print("\n")